## Code Implementation

We've Created Lib using the code , for easy understanding of the diffrent steps on building the libarary there is an example notebook,
that show the build from the start to end. In Addition There is Code Lib with all information from the paper

In [116]:
%pip install sklearn-relief
%pip install pyreadr
%pip install ReliefF
%pip install skfeature-chappers
%pip install scikit_posthocs 

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 9.1/9.1 MB 2.0 MB/s eta 0:00:00
     -------------------------------------- 292.8/292.8 kB 2.6 MB/s eta 0:00:00
     -------------------------------------- 233.7/233.7 kB 2.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [34]:
import pandas as pd
import numpy as np
import itertools
import csv

## Utils

### Fetch Data

We decided to test our implemntation using:
<ul>
    <li>Toy Example [SPECTF.train]</li>
    <li>2 Datasets from ARFF [Lungs Breast]</li>
    <li>2 Datasets from SCV [Lungs Overian]</li>
</ul>
In the creation of the data we made sure all label column are the last for easy acsess for pipline

#### Load Function

In [3]:
def cross_val_methood(df: pd.DataFrame):
    """
        Function we'll return name of division,
        cv number, X,y
    """
    X, y = df[df.columns.difference(['label'])], df['label'].values
    name = None
    n_splits = None
    n_samples = len(X)
    if n_samples < 50: 
        name = "Leave-pair-out"
        n_splits = n_samples // 2
    elif n_samples < 100: 
        name = "leave-one-out"
        n_splits = n_sample
    elif n_samples < 1_000: 
        name = "fold-cv-10"
        n_splits = 10 
    else:
        name = "fold-cv-5"
        n_splits = 5 
    return name, n_splits, X, y

#### Load Function

In [4]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

In [5]:
def load_toy_data() -> ([pd.DataFrame,pd.Series]):
    inner_func = [(f"F{idx+1}R", f"F{idx+1}S") for idx in range(22)]
    columns = ["OVERALL_DIAGNOSIS"] + list(itertools.chain(*inner_func))
    with open("./Data/SPECTF.train") as f:
        lines = f.readlines()
        data_array = np.zeros((len(lines), 45))
        for row_idx, line in enumerate(lines):
            data_array[row_idx] = list(
                map(lambda x: int(x), line.replace("\n", "").split(","))
            )
        data_dict = {col_name: data for col_name, data in zip(columns, data_array)}
        data = pd.DataFrame.from_dict(data_dict) 
        data.rename(columns={data.columns[0]:'label'},inplace=True)
        column_names = data.columns.values
        data.reindex(columns=column_names,copy=False)
        return data

In [6]:
def load_arff_data() -> ([pd.DataFrame,pd.DataFrame]):
    from scipy.io import arff
    parent_path = 'Data/ARFF'
    overian_data = arff.loadarff(f'{parent_path}/Ovarian.arff')
    lungs_data = arff.loadarff(f'{parent_path}/Lung.arff')
    # load files and change label column to class 
    overian_data = pd.DataFrame(overian_data[0])
    lungs_data = pd.DataFrame(lungs_data[0])
    overian_data.rename(columns={overian_data.columns[-1]:'label'},inplace=True)
    lungs_data.rename(columns={lungs_data.columns[-1]:'label'},inplace=True)
    # change label class to True | False
    overian_data['label'] = OrdinalEncoder().fit_transform(overian_data['label'].values.reshape(-1, 1))
    lungs_data['label'] = OrdinalEncoder().fit_transform(lungs_data['label'].values.reshape(-1, 1))
    return overian_data, lungs_data

In [7]:
def load_mat_data() -> ([pd.DataFrame,pd.DataFrame]):
    from scipy.io import loadmat
    parent_path = 'Data/mat'
    YALE_mat = loadmat(f'{parent_path}/YALE.mat')
    ORL_mat = loadmat(f'{parent_path}//ORL.mat')
    # load files and change label column to class 
    YALE_data = pd.DataFrame(YALE_mat['X'])
    YALE_data['label'] = OrdinalEncoder().fit_transform(pd.DataFrame(YALE_mat['Y']).values.reshape(-1, 1))
    ORL_data = pd.DataFrame(ORL_mat['X'])
    ORL_data['label'] = OrdinalEncoder().fit_transform(pd.DataFrame(ORL_mat['Y']).values.reshape(-1, 1))
    return YALE_data,ORL_data

### Let's Check Are Feature Selection On Toy Data

In [8]:
from paper_code import PaperWrapper

In [9]:
toy_df = load_toy_data()
toy_df.head()

,label,F1R,F1S,F2R,F2S,F3R,F3S,F4R,F4S,F5R,...,F18R,F18S,F19R,F19S,F20R,F20S,F21R,F21S,F22R,F22S
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,59.0,72.0,71.0,69.0,70.0,57.0,69.0,61.0,65.0,74.0,...,71.0,70.0,73.0,68.0,68.0,62.0,62.0,59.0,75.0,77.0
2,52.0,62.0,62.0,71.0,66.0,69.0,66.0,60.0,62.0,73.0,...,75.0,66.0,76.0,76.0,64.0,67.0,67.0,68.0,75.0,79.0
3,70.0,69.0,70.0,70.0,61.0,68.0,62.0,60.0,67.0,72.0,...,76.0,66.0,68.0,79.0,65.0,64.0,68.0,69.0,70.0,79.0
4,67.0,67.0,64.0,78.0,66.0,75.0,75.0,62.0,68.0,79.0,...,74.0,68.0,74.0,78.0,68.0,70.0,70.0,67.0,77.0,77.0


As we can see, there is no need to label encode. 
Let's run 3 iteration over Toy.data, inoreder to see the results

In [10]:
X,y = toy_df[toy_df.columns.difference(['label'])].values, toy_df['label'].values
for iter_n in range(3):
    columns_mask = PaperWrapper()(X,y)
    print(columns_mask)

[2 5 8]
[2 5 8]
[2 5 8]


### Algorithem To Compare

In the assignment we were asked to compare are implemntation to 4 diffrece feature selection methood
<ul>
    <li>MRMR</li>
    <li>RELIEF</li>
    <li>SVM</li>
    <li>F CLASSIF</li>
</ul>
Therefore, we need to import this methodes

In [11]:
from skfeature.function.information_theoretical_based.MRMR import mrmr
from sklearn.feature_selection import f_classif, SelectFdr, SelectFromModel,SelectKBest
from sklearn import preprocessing
from sklearn.svm import LinearSVC
import sklearn_relief as relief

In order to compare the data, we'll use 4 dataset, as said in the previous section


In [12]:
overian_df, lungs_df = load_arff_data()
yale_df, orl_df = load_mat_data()

print(
    'Lungs:',lungs_df.shape,
    '\nOverian:',overian_df.shape,
    '\nyALE:',yale_df.shape,
    '\noRL:',orl_df.shape,
)

Lungs: (203, 12601) 
Overian: (253, 15155) 
yALE: (165, 1025) 
oRL: (400, 1025)


In [13]:
overian_df.head()

,MZ-7.86E-05,MZ2.18E-07,MZ9.60E-05,MZ0.000366014,MZ0.000810195,MZ0.001428564,MZ0.002221123,MZ0.003187869,MZ0.004328805,MZ0.005643929,...,MZ19974.404,MZ19977.042,MZ19979.68,MZ19982.319,MZ19984.957,MZ19987.596,MZ19990.235,MZ19992.874,MZ19995.513,label
0,0.494626,0.263735,0.321841,0.220934,0.297622,0.316458,0.154763,0.223685,0.304346,0.241757,...,0.483622,0.449296,0.449296,0.449296,0.449296,0.449296,0.449296,0.449296,0.449296,1.0
1,0.258063,0.406593,0.321841,0.069771,0.333335,0.354432,0.321431,0.144740,0.260869,0.142853,...,0.631765,0.619718,0.619718,0.619718,0.619718,0.619718,0.619718,0.619718,0.619718,1.0
2,0.537636,0.032966,0.321841,0.209307,0.404762,0.113927,0.369049,0.223685,0.536231,0.131865,...,0.038462,0.035918,0.035918,0.035918,0.035918,0.035918,0.035918,0.035918,0.035918,1.0
3,0.000000,0.395605,0.310347,0.197673,0.404762,0.455701,0.416666,0.210527,0.420292,0.274723,...,0.497864,0.486621,0.486621,0.486621,0.486621,0.486621,0.486621,0.486621,0.486621,1.0
4,0.526884,0.395605,0.367817,0.383719,0.488099,0.392405,0.238094,0.500000,0.362316,0.274723,...,0.267096,0.251408,0.251408,0.251408,0.251408,0.251408,0.251408,0.251408,0.251408,1.0


In [14]:
lungs_df.head()

,AFFX-MurIL2_at,AFFX-MurIL10_at,AFFX-MurIL4_at,AFFX-MurFAS_at,AFFX-BioB-5_at,AFFX-BioB-M_at,AFFX-BioB-3_at,AFFX-BioC-5_at,AFFX-BioC-3_at,AFFX-BioDn-5_at,...,101_at,102_at,103_at,104_at,105_at,106_at,107_at,108_g_at,109_at,label
0,-18.600,10.54,0.010,19.440,-16.980,-27.50,-1.600,38.88,-29.120,-42.870,...,8.110,33.210,30.780,47.80,1.630,17.020,13.780,-103.49,76.980,0.0
1,9.120,9.12,10.180,29.290,-4.680,-1.50,-3.620,20.80,-13.180,-35.470,...,15.490,27.170,26.110,45.22,10.180,69.640,-24.850,-34.41,105.730,0.0
2,-2.175,-2.21,-0.060,6.320,-1.775,-16.53,-3.610,16.41,-17.970,-57.020,...,14.610,10.875,10.615,35.14,1.745,29.710,10.465,-42.63,73.735,0.0
3,-1.540,21.75,5.835,23.815,-24.785,-12.89,-4.485,19.50,-21.445,-47.205,...,9.615,27.355,30.860,48.71,10.355,40.845,54.615,-71.38,65.435,0.0
4,-9.070,3.08,-1.980,17.260,-10.090,-15.15,-18.190,13.21,-13.130,-39.470,...,-14.140,23.340,6.120,33.46,-10.090,40.550,16.250,-48.59,39.540,0.0


In [15]:
yale_df.head()

,0,1,2,3,4,5,6,7,8,9,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,label
0,24,24,26,24,29,30,29,22,29,48,...,74,68,53,41,43,48,73,138,205,0.0
1,70,68,49,53,50,50,37,33,26,13,...,175,140,246,255,255,255,248,255,254,0.0
2,74,71,64,62,41,43,57,57,56,58,...,113,213,255,255,241,247,254,253,164,0.0
3,29,31,32,36,33,35,36,37,18,38,...,34,39,39,32,34,31,28,30,30,0.0
4,42,53,50,62,43,53,50,54,54,48,...,246,255,255,254,255,254,254,252,245,0.0


In [16]:
orl_df.head()

,0,1,2,3,4,5,6,7,8,9,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,label
0,75,83,81,75,60,69,71,79,76,91,...,37,40,40,40,38,40,35,40,39,0.0
1,110,120,130,162,171,160,163,164,166,168,...,40,38,39,41,38,36,34,40,33,0.0
2,77,65,49,47,48,90,166,176,117,125,...,44,39,40,41,40,41,38,36,35,0.0
3,48,58,57,45,20,44,53,40,29,31,...,171,166,155,139,149,161,176,184,183,0.0
4,121,119,120,141,141,140,144,143,148,157,...,45,52,47,48,43,43,41,44,40,0.0


### Lets Divide The Run to simple Generato

By using generator we can divide the code logic to simpler division, without a lost in write time.
the data will always return to the original function after row as been created

In [17]:
def multi_pr(y_test,y_score):
    """
        Caculate Metrics from y_test & y_score. 
        The Metrics We Use Are: 
            - PR-AUC
            - AUC
            - ACC
            - MCC
    """
    if len(y_score.shape) == 1:
        precision, recall, _ = precision_recall_curve(y_test,y_score)
        return auc(recall,precision)
    average_precision = dict()
    n_classes = y_score.shape[1]
    precision = dict()
    recall = dict()
    y_test = preprocessing.label_binarize(y_test, classes=list(range(n_classes)))
    # For each class
    for i in range(n_classes):
        precision[i], recall[i], _ = precision_recall_curve(y_test[:, i], y_score[:, i],pos_label=2)
        average_precision[i] = average_precision_score(y_test[:, i], y_score[:, i])
    # A "micro-average": quantifying score on all classes jointly
    precision["micro"], recall["micro"], _ = precision_recall_curve(y_test.ravel(),
        y_score.ravel())
    return average_precision_score(y_test, y_score,average="micro")

In [214]:
def run_model(model,X_train,X_test,y_train,y_test,binary):
    """
        Run Model adnd yield model metrics.
    """
    tic = time.process_time()
    model.fit(X_train,y_train)
    toc = time.process_time()
    yield 'Train Delta', toc-tic
    tic = time.process_time()
    y_prob = model.predict_proba(X_test)
    toc = time.process_time()
    yield 'Test Delta', toc-tic
    y_pred = np.argmax(y_prob,axis=1)
    if binary: 
        y_prob = y_pred
    else:
        n_classes = y_prob.shape[1]
        y_test = preprocessing.label_binarize(y_test, classes=list(range(n_classes)))
    yield 'ACC', accuracy_score(y_test,y_test)
    param =  {'multi_class':'ovo','average':'macro'} if not binary else {}
    temp = np.argmax(y_test,axis=1) if len(y_test.shape)>1 else y_test
    yield 'MCC', matthews_corrcoef(temp,y_pred)
    pr = multi_pr(y_test,y_prob)
    yield 'PR AUC', pr
    try:
        auc_score = roc_auc_score(y_test,y_prob,**param)   
    except ValueError:
        auc_score = 0
    yield 'AUC',auc_score        

In [19]:
def extract_meta(selector,model,X_train,X_test,y_train,y_test,binary):
    """
        Extract Features from db, 
        using Feature Selector Algorithem
    """
    f_out = list(selector.get_feature_names_out())
    mask = selector.get_support()
    support =  list(selector.scores_[mask])
    info = {}
    for metric, val in run_model(model,X_train,X_test,y_train,y_test,binary):
        yield {
            "List of Selected Features Names (Long STRING)": f_out,
            "Selected Features scores": support,
            "Measure Type": metric,
            "Measure Value": ("%.3f" % val),
        }

In [20]:
def run_f_selector(models,selector,df,binary):
    """
        Divide using k-flod, 
        get selector feratures, metrics and calc time
    """
    split_name, cv, X, y = cross_val_methood(df)
    transformer = PowerTransformer()
    X = transformer.fit_transform(X)
    kf = StratifiedKFold(cv)
    for fold, (train_index, test_index) in enumerate(kf.split(X,y)):
        X_train, X_test = X[train_index,:], X[test_index,:]
        y_train, y_test = y[train_index], y[test_index]
        tic = time.process_time()
        selector = selector.fit(X_train,y_train)
        X_train = selector.transform(X_train)
        toc = time.process_time()
        delta = toc - tic
        X_test = selector.transform(X_test)
        for model_name, model, param in models:
            model = model(**param)
            for info in  extract_meta(selector,model,X_train,X_test,y_train,y_test,binary):
                info['Learning algorithm'] = model_name
                info['CV Method'] = split_name
                info['Fold'] = fold
                cpy = info.copy()
                yield info
        cpy["Measure Type"] = "Feature Selection Delta"
        cpy["Measure Value"] = ("%.3f" % delta)
        yield cpy

In [21]:
def run_selector_with_k(k_option,selector,models,df,binary):
    """
        Select k value from list and run on model on db
    """
    for k in k_option:
        selector.k = k
        for info in run_f_selector(models,selector,df,binary):
            info['Number of features selected'] = k
            yield info

In [22]:
def run_all_selectors(k_options,selectors,models,df,binary):
    """
        Run On Selector & trail 
    """
    for selector_name, selector in selectors:
        for info in run_selector_with_k(k_options,selector,models,df,binary):
            info['Filtering Algorithm'] = selector_name
            yield info

In [23]:
def run_all(k_options,selectors,models,df_list):
    with open('results.csv', 'w', encoding='UTF8') as f:
        header = [
            'Dataset Name',
            'Number of samples',
            'Original Number of features',
            'Filtering Algorithm',
            'Learning algorithm',
            'Number of features selected',
            'CV Method',
            'Fold',
            'Measure Type',
            'Measure Value',
            'List of Selected Features Names (Long STRING)',
            'Selected Features scores',
        ]
        writer = csv.writer(f)
        writer.writerow(header)
        for df_name,df in df_list:
            binary = len(df['label'].unique()) == 2
            for info in run_all_selectors(k_options,selectors,models,df,binary):
                info['Dataset Name'] = df_name
                info['Number of samples'] = len(df) 
                info[ 'Original Number of features'] = len(df.columns)  
                line = [info[key] for key in header]
                writer.writerow(line)  

### Create Selector Wrapper

We'll use sklearn SelectKBest and by doing so we've to create wrapper function that will return 
score for each features

In [24]:
wrapper_f_classif = SelectFdr(f_classif,alpha=0.1).score_func
paper_instnace = PaperWrapper()
last_paper_val  = [np.zeros(3),None]

def wrapper_mrmr(X,y):
    index_selected = mrmr(X,y)
    score = np.zeros(len(index_selected))
    for key,val in enumerate(index_selected):
        score[val] = 1/(key+1)
    return score

def wrapper_relief(X,y):
    r = relief.Relief()
    r.fit(X,y)
    return r.w_

def svm_wrapper(X,y):
    lsvc = LinearSVC(C=0.1, penalty="l1", dual=False).fit(X,y)
    return SelectFromModel(lsvc, prefit=True).get_support()

def paper_wrapper(X,y):
    score = np.zeros(X.shape[1])
    if np.array_equal(last_paper_val[0],X):
        return last_paper_val[1]
    last_paper_val[0] = X
    features = paper_instnace(X,y)
    score[features] = 1 
    last_paper_val[1] = score
    return score

In [25]:
k_mrmr = SelectKBest(wrapper_mrmr,k=5)
k_relief = SelectKBest(wrapper_relief,k=5)
k_svm = SelectKBest(svm_wrapper,k=5)
k_f_classif = SelectKBest(wrapper_f_classif,k=5)
k_paper = SelectKBest(paper_wrapper,k=5)

### Create The Running paramaters

In [26]:
from sklearn.metrics import roc_auc_score,accuracy_score,matthews_corrcoef,precision_recall_curve,auc,average_precision_score
from sklearn.preprocessing import PowerTransformer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedKFold
import time

In [27]:
MODELS = [
    ('GN',GaussianNB,{}),
    ('SVM',SVC,{'gamma':'auto','probability':True}),
    ('LR',LogisticRegression,{'random_state':0}),
    ('RF',RandomForestClassifier,{'max_depth':2,'random_state':0}),
    ('KNN',KNeighborsClassifier,{'n_neighbors':3}),
]
FEATURE_SELECTOR = [
    ('f__classif',k_f_classif),
    ('relief',k_relief),
    ('mrmr',k_mrmr),
    ('svm',k_svm),
    ('paper', k_paper),
]
DB_LIST = [
    ('overian',overian_df),
    ('lungs',lungs_df),
    ('yale',yale_df),
    ('orl',orl_df),
]
K_LIST = [1,2,3,4,5,10,15,20,25,30,50,100]

In [ ]:
tmp = run_all(K_LIST,FEATURE_SELECTOR,MODELS,DB_LIST)

After We extrabct all information that cannot be extracted by sklearn pipline.
Now we'll use Pipline.

First We'll create a Feature switcher that can be changed durining gv search gri

In [ ]:
class ClfWrapper(BaseEstimator):


    def __init__(self, estimator = LogisticRegression()):
        self.estimator = estimator
        
    def fit(self, X, y=None, **kwargs):
        self.estimator.fit(X, y)
        return self 

    def predict(self, X, y=None):
        return self.estimator.predict(X)

    def predict_proba(self, X):
        return self.estimator.predict_proba(X)

    def score(self, X, y):
        return self.estimator.score(X, y)
    
    def fit_transform(self,X,y=None):
        return X
    
    def transform(self,X,y=None):
        return X

In [ ]:
class SelectorWrapper(BaseEstimator):


    def __init__(self, estimator = VarianceThreshold(), K=1):
        self.estimator = estimator
        self.estimator.k = K
        
    @property
    def K(self):
        return self.estimator.k 

    @K.setter
    def K(self,K):
        self.estimator.k = K

    def fit(self, X, y=None, **kwargs):
        self.estimator.fit(X, y)
        return self

    def predict(self, X, y=None):
        return self.estimator.predict(X)

    def predict_proba(self, X):
        return self.estimator.predict_proba(X)

    def score(self, X, y):
        return self.estimator.score(X, y)
    
    def fit_transform(X,y=None):
        return self.estimator.fit_transform(X)
    
    def transform(self,X,y=None):
        return self.estimator.transform(X)
    

In [ ]:
def run_on_all(feature_selectors,models,K,dbs):
    pipeline = Pipeline([
        ('preprocess', preprocessor),
        ('selector',SelectorWrapper()),
        ('clf', ClfWrapper()),
    ])
    res = {}
    for df_name, df in dbs:
        parameters = [
        {
            'clf__estimator': models, # SVM if hinge loss / logreg if log loss
            'selector__estimator': feature_selectors, # SVM if hinge loss / logreg if log loss
            'selector__K': K, # SVM if hinge loss / logreg if log loss
        }]
        if df_name not in res:
            res[df_name] = {}
        split_name, cv, X, y = cross_val_methood(df)
        n_class = np.unique(y).shape[0]
        score =  {
            "ACC": 'accuracy',
            "MCC":  make_scorer(matthews_corrcoef),
            "PR-Auc": make_scorer(precision_score),
            # "AUC":   make_scorer(roc_auc_score,needs_proba=True,average='micro',multi_class="ovo"),
        }
        # if n_class == 2: 
        #     score['AUC'] = make_scorer(roc_auc,needs_proba=True,pos_label=2)
        gscv = GridSearchCV(
            pipeline, 
            parameters, 
            scoring=score,
            refit = 'ACC',
            cv=cv, 
            n_jobs=1, 
            return_train_score=False, 
            verbose=3,
        )
        gscv.fit(X,y)
        res[df_name]['results'] = gscv.cv_results_
        res[df_name]['CV'] = split_name
        res[df_name]['feature_names'] = gscv.feature_names_in_
        print(gscv.feature_names_in_)
        
    return res

In [ ]:
# res = run_on_all(
#     FEATURE_SELECTOR,
#     MODELS,
#     K=K_LIST,
#     dbs=DB_LIST
# )

In [117]:
from scipy.stats import friedmanchisquare
import scikit_posthocs as sp

In [354]:
def friedman_test():
    """
    :return:
    """
    i = 0 
    all_aucs = []
    algorithms_and_scores = {}
    df = pd.read_csv('copy.csv')
    grouped = df.groupby(['Dataset Name', 'Filtering Algorithm'])
    
    for name, group in grouped:     
        if i == 6: 
            continue
        i+=1
        aucs = group.loc[group['Measure Type'] == 'AUC' , 'Measure Value']
        acc = group.loc[group['Measure Type'] == 'ACC']  
        # print(type(aucs),len(acc))
        all_aucs.append(aucs.tolist())

        if name[1] not in algorithms_and_scores:
            algorithms_and_scores[name[1]] = []
        algorithms_and_scores[name[1]] = aucs.tolist()
    
    # for auc in all_aucs:
    #     print(len(auc))
    # print(all_aucs)
    fs_result = friedmanchisquare(*all_aucs)
    

    if fs_result.pvalue < 0.05:
        return run_post_hoc(algorithms_and_scores)


def run_post_hoc(algorithms_and_scores):
    """

    :param df:
    :return:
    """
    df = pd.DataFrame.from_dict(algorithms_and_scores)
    long_df = pd.melt(df, var_name='filtering algorithm', value_name='auc score')

    

    ranks_df = df.rank(axis=1, method='max', ascending=False)
    mean = ranks_df.mean(axis=0)
    print('\n')
    # print(mean)
    
    return sp.posthoc_dunn(long_df, val_col='auc score', group_col='filtering algorithm', p_adjust='bonferroni'), mean

In [355]:
post_hoc, mean = friedman_test()

In [356]:
post_hoc

,f__classif,mrmr,paper,relief,svm
f__classif,1.000000e+00,2.560717e-66,4.339892e-84,1.905989e-60,7.608180e-39
mrmr,2.560717e-66,1.000000e+00,2.693817e-01,1.000000e+00,3.719160e-04
paper,4.339892e-84,2.693817e-01,1.000000e+00,2.630494e-02,2.347683e-09
relief,1.905989e-60,1.000000e+00,2.630494e-02,1.000000e+00,8.723088e-03
svm,7.608180e-39,3.719160e-04,2.347683e-09,8.723088e-03,1.000000e+00


In [357]:
mean

f__classif    2.211667
mrmr          3.976667
paper         4.290000
relief        3.816667
svm           3.200000
dtype: float64

In [ ]:
df = pd.read_csv('results.csv')
df1 = df.groupby(['Dataset Name','Fold','Measure Type'])
# print(len(df1))
# for name, group in df1:  
    # aucs = group.loc[group['Measure Type'] == 'AUC' , 'Measure Value']
    # print(len(aucs))
# print(df1.loc[df1['Measure Type'] == 'ACC'],'')
df1 = df.loc[df['Dataset Name'] == 'lungs']
df1.loc[df['Fold'] == 0]

In [ ]:
df = pd.read_csv('results.csv')


grouped = df.groupby(['Dataset Name','Fold','Measure Type'])
ACC = grouped.loc['Measure Type']
df1 = df.loc[df['Dataset Name'] == 'lungs']
